Department of Physics, University of Pisa (AA 2023-2024)

### Multimessenger Physics Laboratory group 2 LOGBOOK


## Analysis of GRB emission 


**30/04/2024-Francesco e Julia**

### informazioni generali

Scegliamo la sorgente GRB090926(181) (le ultime 3 cifre sono le frazioni del giorno)->GRB090926A

Trigger time: FORMATO ISO 2009-09-26 04:20:26.987-> FORMATO MET (Middle European Time) 275631628.987 (secondi dall'inizio missione Fermi, nel 2001)


Abbiamo scoperto dai file TTE :

[GLAST MET] Observation start time : 275631602.065792 ('2009-09-26T04:20:02')

[GLAST MET] Observation stop time : 275631929.966916 ('2009-09-26T04:25:29')

Nel primo HDU osserviamo che ci sono 128R channels.

Nel secondo HDU ci sono invece i tempi e i PHA (Pulse Heights Analysis)

Calcolando l'istogramma di tempi di arrivo dei fotoni, otteniamo la curva di luce del nostro strumento.

**2/05/2024-Francesco, Julia e Chiara**

### light curve

-> come facciamo il binning? risoluzione temporale di 5$\mu$s

-> 3 sigma presenta ancora una coda, che non si capisce bene se sia un afterglow oppure una parte di rumore

-> utilizzare 5 sigma invece seleziona tutto ciò che è che sicuramente segnale e solo segnale! (hard acceptance criteria)

**7/05/2024-Francesco, Julia e Chiara**

### come preparare la nostra postazione nel laboratorio Multimessenger

in terminale

ssh -keygen -t ed25519 -C ""email github""

create-docker

exec-docker 

abbiamo clonato i dati dal repositorio nella cartella work del docker (macchina virtuale)

da terminale:

-heainit (per fare l'inizializzazione di HEASOFT)

-XSPEC

### creiamo bin time file

https://fermi.gsfc.nasa.gov/ssc/data/p6v11/analysis/scitools/gbm_grb_analysis.html

entriamo da terminale nell'environment photon-env-3 e poi nella cartella data/current/TTE

-> il programma non riusciva a leggere il tempo di start e di stop observation dal nostro file interno -> dovevamo risolvere questo problema di metadati

**definiamo il nuovo file txt**

Prendiamo Trigger time (in MET) -> inizio del burst

PRIMA RIGA: background è prima del trigger time, consideriamo  una durata di 10 secondi

SECONDA RIGA: trigger time, e poi diamo una durata di circa 40s (in accordo con la curva di luce cumulativa osservata prima, con un'accettanza di 5 sigma)

**definiamo il nuovo file fits a partire da quello appena creato**

gtbindef

bintype=T

binfile=mytimebinfile.txt

outfile=mytimebinfile.fits

### creiamo i file PHA (per i NaI che hanno fatto il trigger e i BGO) 

Creiamo due file PHA1 per ciascun NaI (uno per il background e uno per i data)

prompt> gtbin

Type of output file (CCUBE|CMAP|LC|PHA1|PHA2) [PHA2] -> single spectrum file

Event data file name[glg_tte_n3_bn080730520.fit]

Output file name[glg_tte_n3_bn080730520.pha]

Spacecraft data file name[NONE]

Algorithm for defining time bins (FILE|LIN|SNR) [FILE]

Name of the file containing the time bin definition[mytimebinfile.fits] 


 selezionare il binning in modo tale da avere un numero minimo di conteggi, con 3 sigma e 5 sigma -> poi passare il flusso per avere barre di errore più piccoli
 
 dummyrsp

**29/05/2024-Francesco, Julia e Chiara**
### spectral fit con XSPEC

-> il nostro GRB e' decisamente puntato verso energie piu' basse<-

->NaI rispondono meglio a band energie<-

->BGO rispondono meglio a powerlaw e non si vede neanche l'energia di picco<-


data [glg_tte_n3_bn080730520.pha]{2}

resp [glg_cspec_n3.rsp]

back [glg_tte_n3_bn080730520.pha]{1}

ign 1-5,100-** (come indicato direttamente da Fermi)

        **per i NaI che hanno attivato il trigger**

model grbm (Band function per GRB continous spectra, normalization costant arbitrary)

$\alpha$=-1, $\delta$=0.02 first power law, with min, bot, top and max as the defaults in XSPEC

$\beta$=-3, $\delta$=0.05 second power law, with min, bot, top and max as the defaults in XSPEC

$E_{peak}$=190, $\delta$=10 with min, bot, top and max as the defaults in XSPEC (temp. in KeV)

        **per i BGO invece**

model powerlaw (normalization costant arbitrary)

$\alpha$=1.5, $\delta$=0.1

        **per somma di una band a basse energia di una band e di una power law ad alte energie **

model grbm+powerlaw (normalization costant arbitrary)

GRBM $\alpha$=-0.66, $\delta$=0.01 -10 -3 3 10

GRBM $\beta$=-2.34, $\delta$=0.1 -10 -5 5 10

GRBM $E_{peak}$=279.5, $\delta$=10 10 50 1000 10000

POWERLAW $\alpha$=0.66, $\delta$=0.01  -10 -3 3 10





fit 1000 (numero di iterazioni)

cpd /Xw

plot data res

setplot energy (si parte da una scala in canali, a una scala fisica di energia)

plot uf res (unfolded model, deconvoluzione dei conteggi dei fotoni prima di entrare nel modello di fit)

iplot

label top [nomeplot]

plot

hardcopy [output_filename.ps/cps]

Test del chi quadro e' piu' affidabile per high photon count. Infatti per avere un effettivo test del chi quadro dovremmo avere per ogni bin una statistica quasi gaussiana

### determinare la fluence

Per il flusso totale dei detector e' necessario quale modello utilizzare per il detector

NaI (Band) 5-300 KeV

BGO (Power Law) 100-1e5 keV


Per la spectral hadness ratio, visto che il tempo di osservazione e' sempre lo stesso, possiamo semplicemente fare il rapportotra i flussi dei NaI detectors (range energetici non presenti nei BGO detectors)

flux 10-50 kev

flux 50-300 kev

flux 2 10 error 100 90 (100 iterazioni per trovare il 90% di confidenza sul flusso da 2 a 10 Kev)

**05/06/2024-Francesco, Julia e Chiara**

### analisi spettrale dei picchi

Abbiamo definito degli txt con gli intervalli del segnale di b0 e n3 che presentano degli evidenti picchi. 

PRIMA RIGA: intervallo  temporale del segnale

SECONDA RIGA: intervallo temporale del background

con gtbindef abbiamo creato da questi file txt i corrispondenti segnali fits

con gtbin abbiamo creato da questi file fits i PHA2 per ogni picco dei detector b0 e n3

abbiamo poi caricato su XSPEC i dati e il background di PHA2 e la matrice di risposta

e abbiamo plotato la forma spettrale dei conteggi intorno al picco

In futuro con piu' tempo a disposizione sarebbe molto interessante lavorare sui fit a powerlaw su questi plot ottenuti. 